In [1]:
import numpy as np
import cv2
import os
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm

/apps/software/standard/core/miniforge/24.3.0-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
pip install --upgrade ipykernel

In [2]:
#Grabs images from Roboflow 

from roboflow import Roboflow
rf = Roboflow(api_key="sYsvlRuqf2HHQYxWNXu5")
project = rf.workspace("alzheimer-h49wu").project("alzheimer-detection-ra7oh")
version = project.version(1)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to alzheimer-detection-1 in folder:: 100%|██████████| 9778/9778 [00:16<00:00, 583.62it/s]
